In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import os
import random
import keras
from keras.models import Sequential
from keras.layers import Conv2D,Dense,Flatten,Dropout,MaxPooling2D, ZeroPadding2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping

from keras.optimizers import Adam, SGD, RMSprop #For Optimizing the Neural Network

from PIL import Image
from PIL import Image, ImageEnhance 
import glob
%matplotlib inline

Using TensorFlow backend.


In [2]:
total_images_train = os.listdir('../input/chest_xray/chest_xray/train/PNEUMONIA/')

In [3]:
image_height = 250
image_width = 250
batch_size = 10
no_of_epochs  = 2

In [4]:
model = Sequential()
model.add(Conv2D(32,(3,3),input_shape=(image_height,image_width,3),activation='relu'))
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(units=128,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=1,activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [5]:
# #This is a Convolutional Artificial Neural Network
# #VGG16 Model
# model = Sequential()
# model.add(ZeroPadding2D((1,1),input_shape=(image_height,image_width,3)))
# model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(ZeroPadding2D((1,1)))
# model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2,2), strides=(2,2)))

# model.add(ZeroPadding2D((1,1)))
# model.add(Conv2D(128, (3, 3), activation='relu'))
# model.add(ZeroPadding2D((1,1)))
# model.add(Conv2D(128, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2,2), strides=(2,2)))

# model.add(ZeroPadding2D((1,1)))
# model.add(Conv2D(256, (3, 3), activation='relu'))
# model.add(ZeroPadding2D((1,1)))
# model.add(Conv2D(256, (3, 3), activation='relu'))
# model.add(ZeroPadding2D((1,1)))
# model.add(Conv2D(256, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2,2), strides=(2,2)))

# model.add(ZeroPadding2D((1,1)))
# model.add(Conv2D(512, (3, 3), activation='relu'))
# model.add(ZeroPadding2D((1,1)))
# model.add(Conv2D(512, (3, 3), activation='relu'))
# model.add(ZeroPadding2D((1,1)))
# model.add(Conv2D(512, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2,2), strides=(2,2)))

# model.add(ZeroPadding2D((1,1)))
# model.add(Conv2D(512, (3, 3), activation='relu'))
# model.add(ZeroPadding2D((1,1)))
# model.add(Conv2D(512, (3, 3), activation='relu'))
# model.add(ZeroPadding2D((1,1)))
# model.add(Conv2D(512, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2,2), strides=(2,2)))

# model.add(Flatten())
# model.add(Dense(4096, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(4096, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(1, activation='softmax'))

# # model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 248, 248, 32)      896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 246, 246, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 123, 123, 32)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 123, 123, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 121, 121, 64)      18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 119, 119, 64)      36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 59, 59, 64)        0         
__________

In [7]:
# train_datagen = ImageDataGenerator(rescale=1./255,
#                                    rotation_range=15,
#                                    shear_range=0.2,
#                                    zoom_range=0.2
#                                    )

# test_datagen = ImageDataGenerator(rescale=1./255)

In [8]:
data = '../input/chest_xray/chest_xray'

train_labels = []
train_files_paths = sorted(glob.glob(data + '/train/PNEUMONIA/*'))
# train_files_paths = train_files_paths[25:335]
for file in train_files_paths:
    temp = file.split('/')[-1]
    if ('bacteria' in temp): train_labels.append(0) # bac = 0
    elif ('virus' in temp): train_labels.append(1) # virus = 1
    else: train_labels.append('error')
            

test_labels = []
test_files_paths = sorted(glob.glob(data + '/test/PNEUMONIA/*'))
# test_files_paths = test_files_paths[25:235]
for file in test_files_paths:
    temp = file.split('/')[-1]
    if ('bacteria' in temp): test_labels.append(0) # bac = 1
    elif ('virus' in temp): test_labels.append(1) # virus = 1
    else: test_labels.append('error')
        
if ('error' in train_labels or 'error' in test_labels): print('Error creating labels!')
else: print('Labels created successfully!')
    

Labels created successfully!


In [9]:
t = train_labels[0]
c0,c1 = 0,0
for i in train_labels:
    if i == t: c0+=1
    else: c1+=1
        
print(len(train_labels),c0,c1,c0+c1)
print('Ligo Unbalanced')

3875 2530 1345 3875
Ligo Unbalanced


In [10]:
# # Load Data

# train_files, test_files = [], []
# for file in train_files_paths:
#     img = cv2.imread(file)
# #     img = np.array(img)
#     img = cv2.resize(img, (image_width, image_height))
#     img = img.astype(np.float32)/255
#     train_files.append(img)
    
# for file in test_files_paths:
#     img = cv2.imread(file)
#     img = cv2.resize(img, (image_width, image_height))
#     img = img.astype(np.float32)/255
#     test_files.append(img)
    
# train_files = np.array(train_files)
# test_files = np.array(test_files)


In [11]:
def change_contrast(img, level):
    factor = (259 * (level + 255)) / (255 * (259 - level))
    def contrast(c):
        return 128 + factor * (c - 128)
    return img.point(contrast)

In [12]:
# Load Data

train_files, test_files = [], []
for file in train_files_paths:
    img = cv2.imread(file)

    #Crop
    y,x=img.shape[0:2]
    if x>y: crimg = img[:,(x-y)//2:x - (x-y)//2,:]
    else:  crimg = img[(y-x)//2:y - (y-x)//2,:,:]
    c0,c1 = crimg.shape[0:2]
    c0,c1 = c0//2,c1//2
    crimg2 = crimg[:,:c0,:]
    #Resize
    img2 = cv2.resize(crimg2, (image_width, image_height))

    #Contrast
    temp = Image.fromarray(img2)
    res = change_contrast(temp, 100)
    res = np.array(res)
    img = res.astype(np.float32)/255
    train_files.append(img)
    
for file in test_files_paths:
    img = cv2.imread(file)

    #Crop
    y,x=img.shape[0:2]
    if x>y: crimg = img[:,(x-y)//2:x - (x-y)//2,:]
    else:  crimg = img[(y-x)//2:y - (y-x)//2,:,:]
    c0,c1 = crimg.shape[0:2]
    c0,c1 = c0//2,c1//2
    crimg2 = crimg[:,:c0,:]

    #Resize
    img2 = cv2.resize(crimg2, (image_width, image_height))


    #Contrast
    temp = Image.fromarray(img2)
    res = change_contrast(temp, 100)
    res = np.array(res)
    img = res.astype(np.float32)/255
    test_files.append(img)
    
    
train_files = np.array(train_files)
test_files = np.array(test_files)

In [13]:
# fil = test_files_paths[27]
# img = cv2.imread(fil)
# y,x=img.shape[0:2]
# if x>y: crimg = img[:,(x-y)//2:x - (x-y)//2,:]
# else:  crimg = img[(y-x)//2:y - (y-x)//2,:,:]
# c0,c1 = crimg.shape[0:2]
# c0,c1 = c0//2,c1//2
# crimg2 = crimg[:,:c0,:]
# img2 = cv2.resize(crimg2, (image_width, image_height))
# plt.imshow(img2)


In [14]:
# plt.imshow(crimg)

In [15]:
# plt.imshow(crimg2)

In [16]:

# im = Image.open(fil)
# # enhancer = ImageEnhance.Contrast(im)
# enhancer = ImageEnhance.Brightness(im)
# # enhancer = ImageEnhance.Sharpness(im)
# enhanced_im = enhancer.enhance(1.0)
# plt.imshow(enhanced_im)

In [17]:
# for file in test_files_paths[10:11]:
#     img = cv2.imread(file)
#     y,x=img.shape[0:2]
#     if x>y: crimg = img[:,(x-y)//2:x - (x-y)//2,:]
#     else:  crimg = img[(x-y)//2:x - (x-y)//2,:,:]
#     img2 = cv2.resize(crimg, (image_width, image_height))
# plt.imshow(img)
# print(img.shape)

In [18]:
# plt.imshow(crimg)


In [19]:


# def change_contrast(img, level):
#     factor = (259 * (level + 255)) / (255 * (259 - level))
#     def contrast(c):
#         return 128 + factor * (c - 128)
#     return img.point(contrast)

# # change_contrast(Image.open('barry.png'), 100)
# test = Image.fromarray(crimg)
# change_contrast(test, 100)



In [20]:
import scipy
from scipy import ndimage
# import matplotlib.pyplot as plt

f = scipy.misc.face(gray=True).astype(float)
blurred_f = ndimage.gaussian_filter(f, 3)

filter_blurred_f = ndimage.gaussian_filter(blurred_f, 1)

alpha = 30
sharpened = blurred_f + alpha * (blurred_f - filter_blurred_f)


In [21]:
# Data Augmentation
train_datagen = ImageDataGenerator(#rescale=1./255,
                                   rotation_range=15,
                                   shear_range=0.2,
                                   zoom_range=0.2
                                   )

test_datagen = ImageDataGenerator()#rescale=1./255)

In [22]:
def lr_schedule(epoch):
    lr = 0.0001
    if epoch > 6:
        lr *= 1e-1
    elif epoch > 80:
        lr = lr/100
    print('Learning rate: ', lr)
    return lr


lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [lr_reducer, lr_scheduler]

In [23]:
# reduce_learning_rate = ReduceLROnPlateau(monitor='loss',
#                                          factor=0.1,
#                                          patience=2,
#                                          cooldown=0,#2,
#                                          min_lr=0.00001,
#                                          verbose=1)

# callbacks = [reduce_learning_rate]

In [24]:
training_set = train_datagen.flow(train_files, train_labels, batch_size=batch_size)

test_set = test_datagen.flow(test_files, test_labels, batch_size=batch_size)

In [25]:
# optimizer = Adam(lr = 0.0001)
# early_stopping_monitor = EarlyStopping(patience = 3, monitor = "val_acc", mode="max", verbose = 2)
# model.compile(loss="sparse_categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)
# history = model.fit_generator(epochs=5, callbacks=[early_stopping_monitor], shuffle=True, validation_data=test_set, generator=training_set, steps_per_epoch=len(train_labels)//batch_size, validation_steps=len(test_labels)//batch_size,verbose=2)
# # prediction = model.predict_generator(generator=train_batches, verbose=2, steps=100)

In [26]:
history = model.fit_generator(training_set, 
#                                validation_data=(test_files,test_labels) ,
                              validation_data= test_set ,
                               steps_per_epoch= len(train_labels)//batch_size, epochs=no_of_epochs*0+20,
                              validation_steps=len(test_labels)//batch_size,
                               callbacks=callbacks) 

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
Learning rate:  0.0001
387/387 [==============================] - 49s 127ms/step - loss: 0.6421 - acc: 0.6535 - val_loss: 0.6028 - val_acc: 0.7410
Epoch 2/20
Learning rate:  0.0001
387/387 [==============================] - 46s 118ms/step - loss: 0.6176 - acc: 0.6711 - val_loss: 0.5656 - val_acc: 0.7564
Epoch 3/20
Learning rate:  0.0001
387/387 [==============================] - 46s 119ms/step - loss: 0.6007 - acc: 0.6956 - val_loss: 0.5631 - val_acc: 0.6564
Epoch 4/20
Learning rate:  0.0001
387/387 [==============================] - 46s 118ms/step - loss: 0.5978 - acc: 0.7018 - val_loss: 0.5195 - val_acc: 0.8051
Epoch 5/20
Learning rate:  0.0001
387/387 [==============================] - 46s 118ms/step - loss: 0.5893 - acc: 0.7041 - val_loss: 0.4941 - val_acc: 0.8205
Epoch 6/20
Learning rate:  0.0001
387/387 [==============================] - 46s 119ms/step - loss: 0.5790 - acc: 0.7155 - val_loss: 0.4870 - val_acc: 0.8077
Epoc

In [27]:
# train_datagen.fit(train_files)
# test_datagen.fit(test_files)
# history = model.fit_generator(train_datagen.flow(train_files, train_labels, batch_size=batch_size,), 
# #                                validation_data=(test_files,test_labels) ,
#                               validation_data= test_datagen.flow(test_files,test_labels) ,
#                                steps_per_epoch= len(train_labels)//batch_size, epochs=2,
#                               validation_steps=len(test_labels)//batch_size,
#                                callbacks=callbacks) 